In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical
from tensorflow import keras
import numpy as np

# CIFAR-10 데이터셋 로드
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train_meta=y_train
# 데이터 전처리

x_train = x_train / 255.
x_test = x_test / 255.

x_train = tf.image.resize(x_train, (32, 32))
x_test = tf.image.resize(x_test, (32, 32))

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [ ]:
np.size(x_train)

153600000

In [71]:
# 모델 구성
def create_model(num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [72]:
# 모델 컴파일
model = create_model(num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 모델 학습
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 104s 131ms/step - loss: 1.4692 - accuracy: 0.4728 - val_loss: 1.1949 - val_accuracy: 0.5809
Epoch 2/10
782/782 [==============================] - 103s 131ms/step - loss: 1.0880 - accuracy: 0.6185 - val_loss: 1.0644 - val_accuracy: 0.6285
Epoch 3/10
782/782 [==============================] - 103s 132ms/step - loss: 0.9358 - accuracy: 0.6740 - val_loss: 0.9550 - val_accuracy: 0.6692
Epoch 4/10
782/782 [==============================] - 101s 129ms/step - loss: 0.8370 - accuracy: 0.7083 - val_loss: 0.8955 - val_accuracy: 0.6865
Epoch 5/10
782/782 [==============================] - 111s 142ms/step - loss: 0.7713 - accuracy: 0.7327 - val_loss: 0.8534 - val_accuracy: 0.7116
Epoch 6/10
782/782 [==============================] - 101s 129ms/step - loss: 0.7033 - accuracy: 0.7572 - val_loss: 0.8529 - val_accuracy: 0.7091
Epoch 7/10
782/782 [==============================] - 99s 127ms/step - loss: 0.6601 - accuracy: 0.7690 - val_loss: 0.8970 - 

In [ ]:
 #모델 평가
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

313/313 [==============================] - 7s 22ms/step - loss: 0.8969 - accuracy: 0.7107
Test Loss: 0.8968533277511597
Test Accuracy: 0.7106999754905701


In [ ]:
#실험

import cv2
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

img_path = '/content/drive/MyDrive/animal/tiger.jpg'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
import numpy as np
# 새로운 데이터 준비

image = tf.io.read_file(img_path)
image = tf.image.decode_jpeg(image, channels=3)  # 컬러 이미지로 디코딩

# 이미지 크기 조정
resized_image = tf.image.resize(image, [32, 32])

# 이미지 정규화 (0~1 범위로 스케일 조정)
resized_image = resized_image / 255.0


# 새로운 레이블 생성

new_data = resized_image
new_data = np.expand_dims(new_data, axis=0)

new_label = 10  
np.shape(new_label)
new_labels = np.expand_dims(new_label, axis=0)
new_labels = np.reshape(new_labels, (1, 1))
last_label = new_label

In [ ]:
np.shape(x_train)

TensorShape([50000, 32, 32, 3])

In [ ]:
np.shape(new_data)

(1, 32, 32, 3)

In [ ]:
np.shape(new_label)

(1, 1)

In [ ]:
np.shape(y_train_meta)

(50000, 1)

In [ ]:
all_labels = np.concatenate([y_train_meta, new_label], axis=0)


50001

In [ ]:
all_labels_t=to_categorical(all_labels, num_classes=11)
all_labels_t.shape[1]

11

In [ ]:
# import numpy as np

# def train_on_new_data(model, x_train, y_train, data, epochs=1):
#     # 새로운 레이블 생성
#     new_laball_labels = np.concatenate([y_train, new_label], axis=0)el = model.predict(data)    # 기존 레이블과 새로운 레이블 결합
#     new_x_train = np.concatenate([x_train, data], axis=0)
#     new_y_train = np.concatenate([y_train, np.array([new_label])], axis=0)

#     # 모델 재학습
#     model.fit(new_x_train, new_y_train, epochs=epochs)


In [76]:
def add_new_label(meta_model, x_train, y_train, new_data, new_labels,last_label, epochs=1):
    # 1. 기존 데이터와 새로운 데이터를 합칩니다.
    

    all_data = np.concatenate([x_train, new_data])
    all_data = all_data / 255.
    all_labels = np.concatenate([y_train, new_labels], axis=0)
    all_labels_t=to_categorical(all_labels, num_classes=last_label+1)
    number_class =all_labels_t.shape[1]

    model = create_model(number_class)



    # 모델 컴파일
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  
    model.fit(all_data, all_labels_t, batch_size=64, epochs=1)
    # 3. 새로운 데이터를 기존 모델에 추가하여 학습합니다.

In [77]:
# 메타러닝 수행: 모델 재학습 및 새로운 레이블 추가
add_new_label(model, x_train, y_train_meta, new_data, new_labels, last_label)

782/782 [==============================] - 93s 118ms/step - loss: 2.1484 - accuracy: 0.2089


In [78]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

313/313 [==============================] - 9s 28ms/step - loss: 2.3129 - accuracy: 0.0858
Test Loss: 2.3128883838653564
Test Accuracy: 0.08579999953508377


In [83]:
test_path = '/content/drive/MyDrive/animal/tiger_test.jpg'

test_image = tf.io.read_file(test_path)
test_image = tf.image.decode_jpeg(test_image, channels=3)  # 컬러 이미지로 디코딩

# 이미지 크기 조정
test_image = tf.image.resize(test_image, [32, 32])

# 이미지 정규화 (0~1 범위로 스케일 조정)
test_image = test_image / 255.0

tiger_test = tf.expand_dims(test_image, axis=0)  # 배치 차원 추가

predictions = model.predict(tiger_test)
predictions

1/1 [==============================] - 0s 36ms/step


array([[0.11310343, 0.09475081, 0.08845346, 0.11490147, 0.07273505,
        0.09777839, 0.09019503, 0.10205104, 0.11440457, 0.11162667]],
      dtype=float32)

In [85]:
predicted_class = np.argmax(predictions)
predicted_class

3